## 0. 환경설정

In [2]:
!pip install langchain_openai
!pip install langchain_community

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## 1. Langchain 제공 기능

참고 자료 : https://python.langchain.com/v0.2/docs/how_to/chatbots_memory/

### 1) Message passing
- 이전 대화 내용을 chain에 넘겨주는 것

- The simplest form of memory is simply passing chat history messages into a chain.

In [3]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o") # gpt-3.5-turbo-0125

In [4]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{messages}"),
    ]
)

chain = prompt | chat

ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.",
            ),
            ("ai",  "いただきます (이타다키마스)"),
            ("human", "내가 방금 뭐라고 했지?"),
        ],
    }
)
print(ai_msg.content)

당신은 "잘 먹겠습니다"를 일본어로 어떻게 말하는지 물어보셨습니다. 이에 대한 일본어 표현은 いただきます (이타다키마스)입니다.


In [5]:
ai_msg = chain.invoke(
    {
        "messages": [
            (
                "human",
                "내 이름은 승룡킴이고, AI를 공부하고 있어.",
            ),
            ("ai",  "그렇군요. 흥미로운데요?"),
            ("human", "내 이름이 뭐라고?"),
        ],
    }
)
print(ai_msg.content)

당신의 이름은 승룡킴입니다. AI 공부를 하고 계시는군요!


- 응용 : 대화 쌓일 때마다 히스토리 저장하도록 구현

In [6]:
history_list = []
while(True):
    user_input = input()
    if user_input == "종료": break
    history_list.append(
        (
            "human",
            user_input,
        )
    )
    print("## CHAT_HISTORY ##")
    print(history_list, "\n")
    ai_msg = chain.invoke(
        {
            "messages": history_list,
        }
    )
    print("AI Says : ",ai_msg.content)

    history_list.append(
        (
            "ai",
            ai_msg.content,
        )
    )

## CHAT_HISTORY ##
[('human', '얘야')] 

AI Says :  무엇을 도와드릴까요?
## CHAT_HISTORY ##
[('human', '얘야'), ('ai', '무엇을 도와드릴까요?'), ('human', '안녕하니')] 

AI Says :  안녕하세요! 어떻게 도와드릴까요?
## CHAT_HISTORY ##
[('human', '얘야'), ('ai', '무엇을 도와드릴까요?'), ('human', '안녕하니'), ('ai', '안녕하세요! 어떻게 도와드릴까요?'), ('human', '저메추')] 

AI Says :  오늘 날씨와 기분에 따라 달라질 수 있겠지만, 간단하고 든든하게 김치찌개는 어떠세요? 아니면 외식을 고려하신다면, 근처에 맛있는 한식당이나 일식당을 찾아보는 것도 좋은 선택일 것 같아요. 어떤 걸 드시고 싶으신가요?
## CHAT_HISTORY ##
[('human', '얘야'), ('ai', '무엇을 도와드릴까요?'), ('human', '안녕하니'), ('ai', '안녕하세요! 어떻게 도와드릴까요?'), ('human', '저메추'), ('ai', '오늘 날씨와 기분에 따라 달라질 수 있겠지만, 간단하고 든든하게 김치찌개는 어떠세요? 아니면 외식을 고려하신다면, 근처에 맛있는 한식당이나 일식당을 찾아보는 것도 좋은 선택일 것 같아요. 어떤 걸 드시고 싶으신가요?'), ('human', '')] 

AI Says :  궁금한 점이나 도움이 필요하시면 언제든지 말씀해 주세요!


### 2) Chat history
- ChatMessageHistory() 클래스 사용하기
- It's perfectly fine to store and pass messages directly as an array, but we can use LangChain's built-in message history class to store and load messages as well.

https://api.python.langchain.com/en/latest/chat_message_histories/langchain_community.chat_message_histories.in_memory.ChatMessageHistory.html

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory

chat_history = ChatMessageHistory()

chat_history.add_user_message(
    "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘."
)

chat_history.add_ai_message("いただきます (이타다키마스)")

chat_history.messages
# chat_history.clear()

[HumanMessage(content="일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='いただきます (이타다키마스)', additional_kwargs={}, response_metadata={})]

- 대화내용 바로바로 저장하기

In [8]:
chat_history = ChatMessageHistory()

################ USER INPUT - 1 ################
input1 = "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘."

# 대화 히스토리에 입력(user_input) 저장
chat_history.add_user_message(input1)
print("첫번째 입력 후 history : ", chat_history.messages)

################ AI RESPONSE - 1 ################
# 답변 생성(response)
response = chain.invoke(
    {
        "messages": chat_history.messages,
    }
)

# 대화 히스토리에 답변(response) 저장
chat_history.add_ai_message(response)
print("첫번째 답변 후 history : ", chat_history.messages)
print()

################ USER INPUT - 2 ################
input2 = "내가 방금 뭐라고 물어봤지?"

# 대화 히스토리에 입력(user_input) 저장
chat_history.add_user_message(input2)
print("두번째 입력 후 history : ", chat_history.messages)

################ AI RESPONSE - 2 ################
# 답변 생성(response)
print("[두번째 입력에 대한 답변]")
chain.invoke(
    {
        "messages": chat_history.messages,
    }
)

첫번째 입력 후 history :  [HumanMessage(content="일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.", additional_kwargs={}, response_metadata={})]
첫번째 답변 후 history :  [HumanMessage(content="일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘.", additional_kwargs={}, response_metadata={}), AIMessage(content='일본어로 \'잘 먹겠습니다.\'는 "いただきます"라고 합니다. 한글 발음으로는 "이타다키마스"입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 59, 'total_tokens': 93, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e413f45763', 'id': 'chatcmpl-CnLfOnGiJIoFO9aHXp37aqWzQx1wD', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b267e-abe4-77c3-bdee-f9fadf5f4f80-0', usage_metadata={'input_tokens': 59, 'output_to

AIMessage(content="당신은 일본어로 '잘 먹겠습니다.'를 어떻게 말하는지, 그리고 그에 대한 한글 발음도 함께 알려달라고 물어보셨습니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 112, 'total_tokens': 148, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e413f45763', 'id': 'chatcmpl-CnLfPdW3hYIiT1G7iXt4tnndKJSfh', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b267e-b087-7501-beef-9120b241702e-0', usage_metadata={'input_tokens': 112, 'output_tokens': 36, 'total_tokens': 148, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

- 응용 : 대화 쌓일 때마다 히스토리 저장하도록 구현

In [9]:
chat_history = ChatMessageHistory()

while(True):
    user_input = input()
    if user_input == "종료": break
    chat_history.add_user_message(user_input)
    response = chain.invoke(
        {
            "messages": chat_history.messages,
        }
    )
    chat_history.add_ai_message(response)
    print(response.content)

안녕하세요! 무엇을 도와드릴까요?
만나서 반가워요! 어떻게 도와드릴까요?
힌트를 좀 주실 수 있을까요? 제가 맞춰볼게요!
아, 승룡킴님이시군요! 만나서 반갑습니다. 오늘 어떻게 도와드릴까요?
승룡킴님이라고 하셨죠! 맞습니까?
가드 포지션에서 농구를 잘하기 위해서는 여러 가지 기술과 능력을 발전시켜야 합니다. 다음은 몇 가지 중요한 사항들입니다:

1. **핸들링 능력**: 드리블 실력을 향상시키세요. 양손을 자유자재로 사용할 수 있도록 연습하고, 다양한 드리블 기술을 배우세요.

2. **슛 정확도**: 미들슛과 3점슛 모두 안정적으로 넣을 수 있도록 슈팅 연습을 많이 하세요. 언제든지 정확하게 슛할 수 있도록 하세요.

3. **패스 능력**: 다양한 패스 기술을 익히고, 동료들에게 정확하고 빠르게 패스를 전달할 수 있는 능력을 키우세요.

4. **코트 비전**: 경기장에서 전체적인 상황을 파악하고, 언제 패스를 해야 하는지, 언제 드리블을 해야 하는지를 결정할 수 있는 시야를 길러야 합니다.

5. **수비 기술**: 좋은 수비는 공격 못지않게 중요합니다. 발빠르게 움직이고, 상대방의 움직임을 예측하는 능력을 키우세요.

6. **게임 조율 능력**: 상황 판단력이 뛰어나야 합니다. 어떤 플레이를 해야 하는지, 팀을 어떻게 이끌어야 하는지를 계산할 수 있어야 합니다.

7. **체력과 민첩성**: 충분한 체력과 빠른 발이 필수적입니다. 꾸준한 체력 훈련과 스피드 훈련을 통해 이를 강화하세요.

8. **멘탈**: 침착함과 자신감을 가지세요. 실수해도 빠르게 회복하고, 팀을 믿고 협력하는 자세가 중요합니다.

꾸준한 연습과 경험이 실력 향상에 큰 도움이 될 것입니다. 노력과 열정으로 좋은 성과를 이루시길 바랍니다!


### 3) Automatic history management
- RunnableWithMessageHistory() 활용하여 메모리 관리하기
- The previous examples pass messages to the chain explicitly. This is a completely acceptable approach, but it does require external management of new messages. LangChain also includes an wrapper for LCEL chains that can handle this process automatically called RunnableWithMessageHistory.

In [10]:
chat = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ]
)

chain = prompt | chat

In [11]:
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_history_for_chain = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain, # 실행할 Runnable 객체
    lambda session_id: chat_history_for_chain, # 세션 기록을 가져오는 함수
    input_messages_key="input", # 입력 메시지의 Key
    history_messages_key="chat_history", # 대화 히스토리 메시지의 Key
)

In [12]:
chain_with_message_history.invoke(
    {"input": "일본어로 '잘 먹겠습니다.' 어떻게 말해? 한글 발음도 함께 말해줘."},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='일본어로 \'잘 먹겠습니다.\'는 "いただきます"라고 합니다. 한글 발음은 "이타다키마스"입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 59, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e413f45763', 'id': 'chatcmpl-CnLhd1LKlBeTD6dDv9Y4AmNIcgsXl', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2680-ca02-7a60-8cb0-c98edbdf69b7-0', usage_metadata={'input_tokens': 59, 'output_tokens': 33, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
chain_with_message_history.invoke(
    {"input": "내가 뭐라고 물어봤지?"}, {"configurable": {"session_id": "unused"}}
)

AIMessage(content='당신은 "잘 먹겠습니다."를 일본어로 어떻게 말하는지, 그리고 한글 발음도 함께 알려달라고 물었습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 109, 'total_tokens': 139, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e413f45763', 'id': 'chatcmpl-CnLhnZxwr1y1Tutw9fWnAFsMlHtbz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2680-f1e2-7820-b115-a7c94564f6dc-0', usage_metadata={'input_tokens': 109, 'output_tokens': 30, 'total_tokens': 139, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

히스토리 저장 함수를 따로 호출할 필요없이 invoke 실행만으로 대화 기록이 자동 저장되는 것을 확인 가능!

### 4) Modifying chat history
- 이전 대화 내용을 가공하여 넘겨주기
- Modifying stored chat messages can help your chatbot handle a variety of situations.

In [14]:
chat_history = ChatMessageHistory()

chat_history.add_user_message("안녕하세요. 제 이름은 나연입니다.")
chat_history.add_ai_message("안녕하세요, 나연님! 무엇을 도와드릴까요?")
chat_history.add_user_message("날씨 좋은 날 들을만 한 노래 추천해주세요.")
chat_history.add_ai_message("Carpenters - Close to you 를 추천해요.")

chat_history.messages
# chat_history.clear()

[HumanMessage(content='안녕하세요. 제 이름은 나연입니다.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='안녕하세요, 나연님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='날씨 좋은 날 들을만 한 노래 추천해주세요.', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Carpenters - Close to you 를 추천해요.', additional_kwargs={}, response_metadata={})]

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability. The provided chat history includes facts about the user you are speaking with. YOU MUST ANSWER IN KOREAN.",
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [17]:
from langchain_core.runnables import RunnablePassthrough

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "Distill the above chat messages into a single summary message. Include as many specific details as you can.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True

chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

- summarize_messages 함수의 실행 결과를 messages_summarized 라는 이름의 Key 로 저장

    - 정의한 chain 안에 "messages_summarized" Key 가 정의되고 있지 않으므로, 이는 `summarize_messages` 함수를 실행하기 위한 무시 가능한 Key
    - 함수가 실행되면 chat_history 에 요약된 히스토리가 저장됨


In [18]:
chain_with_summarization.invoke(
    {"input": "제 이름을 기억하고 있나요?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='네, 나연님이시죠! 어떻게 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 103, 'total_tokens': 119, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e413f45763', 'id': 'chatcmpl-CnLlVFM1xkQoOYgk1xNrthA3TUxT8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2684-741f-7f83-9341-5dfd6611cfde-0', usage_metadata={'input_tokens': 103, 'output_tokens': 16, 'total_tokens': 119, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [19]:
chain_with_summarization.invoke(
    {"input": "그 가수는 남자인가요 여자인가요?"},
    {"configurable": {"session_id": "unused"}},
)

AIMessage(content='카펜터스(The Carpenters)는 남매 듀오로, 리처드 카펜터와 카렌 카펜터로 구성되어 있습니다. 카렌 카펜터는 여성 보컬리스트로, 매우 아름다운 목소리를 가지고 있었습니다. 리처드 카펜터는 주로 피아노와 백업 보컬을 담당했습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 124, 'total_tokens': 206, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_e413f45763', 'id': 'chatcmpl-CnLmJgotiDfNrUIUwwPZCaWXutlok', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b2685-3754-7b93-8194-6877a2481a57-0', usage_metadata={'input_tokens': 124, 'output_tokens': 82, 'total_tokens': 206, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## 2. 끝말잇기 게임 구현하기

- 4) Modifying chat history 의 활용!

In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough
# 대화 기록을 저장할 히스토리 클래스 불러오기
chat_history = ChatMessageHistory()

# chat_history.add_user_message("끝말잇기 하자")
# chat_history.add_ai_message("좋습니다. 제가 먼저 시작할게요. 바나나!")
# chat_history.add_user_message("나이테")
# chat_history.add_ai_message("테이프")

chat_history.messages

[]

In [21]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """당신은 끝말잇기 게임을 진행하는 AI 챗봇입니다. 아래는 게임 규칙입니다. 당신과 user 의 입력에서 아래 규칙이 꼭 지켜져야 하며, 지키지 않은 사람에게 패배를 알린 뒤, 끝말잇기 게임을 종료합니다.
                1. 주어진 대화 기록에서 이미 나왔던 단어를 다시 말했을 경우 패배합니다.
                2. 두음법칙을 허용합니다. (ex. 리 -> 이, 력 -> 역, 락 -> 낙)
                3. 국어사전에 존재하는 단어이자, 명사여야 합니다.
            """,
        ),
        ("placeholder", "{chat_history}"),
        ("user", "{input}"),
    ]
)

chain = prompt | chat

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [22]:
def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),
            (
                "user",
                "위 채팅 메시지는 끝말잇기 게임을 진행한 대화내용입니다. 언급한 단어들만 나열하여 저장해주세요.",
            ),
        ]
    )
    summarization_chain = summarization_prompt | chat

    # chat_history 에 저장된 대화 기록을 요약프롬프트에 입력 & 결과 저장
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})

    # chat_history 에 저장되어있던 기록 지우기
    chat_history.clear()

    # 생성된 새로운 요약내용으로 기록 채우기
    chat_history.add_message(summary_message)

    return True


chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

In [23]:
while(True):
    user_input = input("🙋‍♂️ YOUR TURN : ")
    print(" YOU : ", user_input)
    if user_input == "종료": break
    response = chain_with_summarization.invoke(
                {"input": user_input},
                {"configurable": {"session_id": "unused"}},
            )
    print("✍ AI TURN : ", response.content)

 YOU :  사과
✍ AI TURN :  과일
 YOU :  일상
✍ AI TURN :  상점
 YOU :  점무늬
✍ AI TURN :  늬우스
 YOU :  스카우트
✍ AI TURN :  트로피카나
 YOU :  나쁘다
✍ AI TURN :  죄송하지만 "나쁘다"는 명사가 아닙니다. 명사를 사용해 주세요.
 YOU :  나이테
✍ AI TURN :  테니스. 다음 단어를 이어주세요!
 YOU :  스레빠
✍ AI TURN :  "빠"로 시작하는 단어로 이어가겠습니다. 

빠르기

이제 "기"로 시작하는 단어를 말씀해 주세요.
 YOU :  기러기
✍ AI TURN :  "기러기" 좋습니다! 다음 단어는 "기"로 시작해야 합니다. 

기차
 YOU :  차표한잔
✍ AI TURN :  "차표한잔"은 사전에 등재된 명사가 아닙니다. 제가 패배했습니다. 끝말잇기 게임을 종료합니다. 추가로 궁금한 점이 있으시면 말씀해 주세요!
 YOU :  종료
